#### Adopte1Arbre - notebook - dataset exploration

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

import logging
logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

import dtale
import dtale.global_state as global_state
global_state.set_chart_settings({'scatter_points': 20000, '3d_points': 40000})

import os
import json

In [4]:
os.chdir(r'C:\Users\Jean\Documents\GitHub\Adopte1Arbre')

In [8]:
df=pd.read_csv('les-arbres-plantes.csv', sep=';')

In [9]:
dtale.show(df)

#### review every columns  
- 5734 arbres plantés
- créer des catégories : "tout petit tout mignon", "à taille humaine", "un grand spécial calin" à partir de la circonférence et de la hauteur
- un arbre de 110 m ???
- les rares (<100)

In [10]:
df.shape

(5734, 14)

In [11]:
df["Emplacement - Arrondissement"].value_counts()

PARIS 13E ARRDT      610
PARIS 12E ARRDT      545
PARIS 16E ARRDT      475
PARIS 15E ARRDT      436
PARIS 20E ARRDT      384
SEINE-SAINT-DENIS    374
PARIS 14E ARRDT      368
PARIS 18E ARRDT      354
PARIS 19E ARRDT      260
PARIS 11E ARRDT      244
PARIS 17E ARRDT      242
PARIS 7E ARRDT       234
PARIS 8E ARRDT       229
VAL-DE-MARNE         197
PARIS 10E ARRDT      149
HAUTS-DE-SEINE       139
PARIS 5E ARRDT        98
BOIS DE BOULOGNE      87
BOIS DE VINCENNES     67
PARIS 4E ARRDT        59
PARIS 6E ARRDT        56
PARIS 9E ARRDT        50
PARIS 1ER ARRDT       27
PARIS 3E ARRDT        27
PARIS 2E ARRDT        23
Name: Emplacement - Arrondissement, dtype: int64

In [13]:
df["Arbre - Circonférence en cm"].value_counts()

20.0     4399
15.0      299
10.0      285
25.0      268
14.0       80
16.0       67
30.0       56
18.0       53
5.0        33
35.0       28
0.0        25
23.0       20
22.0       16
12.0       13
21.0        9
6.0         7
26.0        6
24.0        5
40.0        4
27.0        4
45.0        4
75.0        4
80.0        3
28.0        3
19.0        3
13.0        3
31.0        2
50.0        2
70.0        2
17.0        2
59.0        2
100.0       2
65.0        2
4.0         2
3.0         2
1.0         2
55.0        2
185.0       1
95.0        1
90.0        1
46.0        1
60.0        1
110.0       1
135.0       1
128.0       1
97.0        1
160.0       1
2.0         1
67.0        1
140.0       1
130.0       1
113.0       1
Name: Arbre - Circonférence en cm, dtype: int64

In [14]:
df["Arbre - Hauteur en m"].value_counts()

5.0      4450
3.0       539
4.0       317
2.0       161
1.0       118
6.0        87
0.0        25
7.0        12
12.0        7
15.0        6
10.0        4
11.0        2
17.0        2
110.0       1
9.0         1
23.0        1
8.0         1
Name: Arbre - Hauteur en m, dtype: int64

In [25]:
df["Arbre Essence - Nom français"].value_counts().loc["Platane"]

923

#### fonctions de préparation de filtres

In [36]:
def rareOupas(df):
    df_ok=df.copy().dropna(subset=["Arbre Essence - Nom français"], axis=0)
    df_Essence=df["Arbre Essence - Nom français"].value_counts()
    df_ok["rareOupas"]=["rare" if df_Essence.loc[df_ok["Arbre Essence - Nom français"].iloc[i]]<100 else "frequent" for i in range(df_ok.shape[0])]
    return df_ok

In [43]:
def classTaille(df):
    df_ok=df.copy()
    bins = [0, 3, 5, 10,20]
    labels = ["petit","moyen","grand", "geant"]
    df_ok['classeTaille'] = pd.cut(df_ok["Arbre - Hauteur en m"], bins=bins, labels=labels)
    return df_ok

In [44]:
df_ok=rareOupas(df)

In [38]:
df_ok.shape

(5631, 15)

In [39]:
df_ok

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,2040728,DASCO,PARIS 18E ARRDT,NaN,ECOLE ELEMENTAIRE / 33 RUE DE L EVANGILE,20.0,5.0,australis,NaN,Celtis,2023-03-01T02:00:00+01:00,Micocoulier,"{""coordinates"": [2.3650679085758384, 48.894450...","48.894450977140366, 2.3650679085758384",frequent
5729,199643,DJS,PARIS 20E ARRDT,BOULODROME,CENTRE SPORTIF DOCTEURS DEJERINE / 36 RUE DES ...,20.0,5.0,campestre,''Elsrijk'',Acer,2022-02-21T02:00:00+01:00,Erable,"{""coordinates"": [2.412440607282452, 48.8552224...","48.8552224410884, 2.412440607282452",frequent
5730,268337,Alignement,PARIS 11E ARRDT,8,RUE DU GENERAL RENAULT,20.0,5.0,hippocastanum,''Baumannii'',Aesculus,2022-01-13T02:00:00+01:00,Marronnier,"{""coordinates"": [2.378364905304499, 48.8612889...","48.861288928546706, 2.378364905304499",frequent
5731,2031599,Alignement,PARIS 5E ARRDT,NaN,PORT DE LA TOURNELLE,20.0,5.0,alba,''Raket'',Populus,2022-02-14T02:00:00+01:00,Peuplier,"{""coordinates"": [2.3521724753846245, 48.851115...","48.851115269136265, 2.3521724753846245",rare


2023-08-04 12:17:29,422 - INFO     - Executing shutdown due to inactivity...
2023-08-04 12:17:41,767 - INFO     - Executing shutdown...
2023-08-04 12:17:41,773 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "C:\Users\Jean\anaconda3\envs\pu_env\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jean\anaconda3\envs\pu_env\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jean\anaconda3\envs\pu_env\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jean\anaconda3\envs\pu_env\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Jean\anaconda3\envs\pu_env\lib\site-packages\dtale\app.py", line 440, in shutdown
    shutdown_server()
  File "C:\User

In [45]:
df_ok=classTaille(df_ok)

In [46]:
df_ok.head()

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas,classeTaille
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare,petit
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare,moyen
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent,moyen
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent,moyen
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent,moyen


In [67]:
df_ok["coordGeo"]=[[float(my_val) for my_val in df["geo_point_2d"].iloc[i].split(",")] for i in range(df_ok.shape[0])]
df_ok["lat"]=[float(df["geo_point_2d"].iloc[i].split(",")[0]) for i in range(df_ok.shape[0])]
df_ok["lon"]=[float(df["geo_point_2d"].iloc[i].split(",")[1]) for i in range(df_ok.shape[0])]
df_ok.to_csv('les-arbres-plantes-clean.csv', sep=';')

In [68]:
df_ok.head()

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas,classeTaille,coordGeo,lat,lon
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare,petit,"[48.84378486022892, 2.3896969976249585]",48.843785,2.389697
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare,moyen,"[48.88748235172613, 2.313351848419123]",48.887482,2.313352
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent,moyen,"[48.90653833095531, 2.3509489697880204]",48.906538,2.350949
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent,moyen,"[48.825376731279526, 2.325937752354315]",48.825377,2.325938
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent,moyen,"[48.89162264234357, 2.345930879134525]",48.891623,2.345931


In [64]:
type(df_ok["coordGeo"].iloc[0][0])

float